# Import

In [13]:
from google.colab import drive
mydrive = '/content/drive/'
drive.mount(mydrive)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import cv2, numpy as np


In [15]:
!pip uninstall tensorflow -y
!pip install tensorflow==1.14.0
%tensorflow_version 1.x
import tensorflow

Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
  Using cached https://files.pythonhosted.org/packages/de/f0/96fb2e0412ae9692dbf400e5b04432885f677ad6241c088ccc5fe7724d69/tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl


TensorFlow is already loaded. Please restart the runtime to change versions.


# VGG16 Structure Overview


In [16]:
m = VGG16(weights='imagenet')
print(m.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

# Data preprocessing


In [0]:
import glob, os
from keras.preprocessing.image import img_to_array, load_img

# TODO: load image from all subdirectory
# convert the image pixels to a numpy array 
# reshape using load_img
# def loading_scaled_data(path):
#   imgs = [img_to_array(load_img(file, target_size=(224, 224))) for file \
#       in glob.glob(path)]
#   return imgs



Load data

In [0]:
def loading_all_data(img_path):
    imgs = []
    level = []
    class_num = 0
    for sub_dict in os.listdir(img_path):
        path = img_path + sub_dict + '/*.jpg'
        imgs = imgs + [img_to_array(load_img(img, target_size=(224, 224))) \
                       for img in glob.glob(path)]
        level = level + [sub_dict for img in glob.glob(path)]
        class_num = class_num + 1 
    return imgs ,level ,class_num

In [0]:
img_path = mydrive + 'My Drive/Project/0425guava-classified/'
X, y ,class_num= loading_all_data(img_path)
# 0,3,4:body
# 1:bottom
# 2:head

In [20]:
print(y[500])

4


Split

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test ,y_train , y_test = train_test_split( X, y, test_size=0.2, random_state=42)

# 1/5 images for test

In [22]:
"""
for img in x_train:
    print(img.shape)
    plt.imshow(img.astype(np.uint8))
    plt.show()
    # img = np.expand_dims(img, axis=0)
    print(img.shape)
"""

'\nfor img in x_train:\n    print(img.shape)\n    plt.imshow(img.astype(np.uint8))\n    plt.show()\n    # img = np.expand_dims(img, axis=0)\n    print(img.shape)\n'

In [0]:
# import tensorflow as tf
# x_train = np.expand_dims(x_train, axis=0)
# x_train = tf.squeeze(x_train)
# not affected

(discarded)

In [0]:
from keras.utils import to_categorical
y_train_hot_encoded = to_categorical(y_train)
y_test_hot_encoded = to_categorical(y_test)

In [25]:
"""
print(np.array(x_train).shape)
print(y_train_hot_encoded.shape)

print(np.array(x_test).shape)
print(y_train_hot_encoded[0])
print(np.array(y_train_hot_encoded).shape)
"""

'\nprint(np.array(x_train).shape)\nprint(y_train_hot_encoded.shape)\n\nprint(np.array(x_test).shape)\nprint(y_train_hot_encoded[0])\nprint(np.array(y_train_hot_encoded).shape)\n'

# VGG16 finetune



Extract/Add certain layer

In [0]:
from keras.layers.core import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers.convolutional import MaxPooling2D

In [0]:
# The top layer of the network uis the last passing layer.
# include_top = false: remove the classification layer to define yours

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))




# get layers 
x = base_model.output

# and add average pooling layer?
feature_out = MaxPooling2D()(x)



# add fully-connected layer
x = Dense(4096, activation='relu', name='fc1')(feature_out)

x = Dense(1024, activation='softmax', name='fc2' )(x)
# 7 levels
predictions = Dense(class_num, activation='softmax', name='predictions')(x)

model = Model(inputs=[base_model.input], outputs=[feature_out, predictions])


In [28]:
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

Lock all layer before compiling.\
In transfer learning, we reuse model by **freezing** or **finen tuning** the layers of a model.

In [0]:
# freeze pre-trained model area's layer
for layer in base_model.layers:
    layer.trainable = False

Learning rate (not used)

In [0]:
from keras.callbacks import LearningRateScheduler
import math
# learning rate schedule
def step_decay(epoch):
  initial_lrate = 0.1
  drop = 0.5
  epochs_drop = 10.0
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrate = LearningRateScheduler(step_decay)

 optimizer

In [0]:
from keras.optimizers import SGD
# update the weight that are added
sgd = SGD(lr=0.01, decay=0.0001, momentum=0.9, nesterov=True)


custom loss

Pass optimizer, loss function, and some metrics to model

In [0]:
model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])

# Train

In [39]:
# model.fit_generator(...)


hitory = model.fit( [np.array(x_train), np.array(y_train_hot_encoded)], \
          epochs=20, batch_size = 32, \
          validation_data=(np.array(x_test), np.array(y_test_hot_encoded)), \
          callbacks=[lrate])
model.save('/content/drive/My Drive/Project/saved_model/model_test.h5') 

ValueError: ignored

In [0]:
# model.fit_generator(...)


hitory = model.fit(np.array(x_train),\
          epochs=20, batch_size = 32, \
          validation_data=(np.array(x_test), np.array(y_test_hot_encoded)), \
          callbacks=[lrate])
model.save('/content/drive/My Drive/Project/saved_model/model_test.h5') 

In [0]:
# model.fit_generator(...)


hitory = model.fit(np.array(x_train),np.array(x_train),\
          epochs=20, batch_size = 32, \
          validation_data=(np.array(x_test), np.array(x_test)), \
          callbacks=None)
model.save('/content/drive/My Drive/Project/saved_model/model_test.h5') 

verbose：日誌顯示
> verbose = 0為不在標準輸出流輸出日誌信息\
verbose = 1為輸出進度條記錄\
verbose = 2為每個epoch輸出一行記錄









In [0]:

plt.plot(hitory.history['acc'])
plt.plot(hitory.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(hitory.history['loss'])
plt.plot(hitory.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict

In [0]:
i = np.expand_dims(imgs[3], axis=0)

In [0]:
i = preprocess_input(i )
res = model.predict(i)
print(res.shape)

In [0]:
from keras.applications.vgg16 import decode_predictions

# print('Predicted:', decode_predictions(res, top=3)[0])

# convert the probabilities to class labels
label = decode_predictions(res)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

# =============================================================